## Import


In [1]:
import gym
import random
from IPython.display import clear_output
import numpy as np

In [2]:
env = gym.make('Taxi-v3').env

# tipo de açoes 
# 0 = south 1 = north 2 = east 3 = west 4 = pickup 5 = dropoff

## Treinamento

In [4]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [5]:
%%time
alpha = 0.1
gamma = 0.6
epsilon = 0.1

for i in range(10000):
    estado = env.reset()
    
    penalidades, recompensa =0,0
    done = False
    while not done:
        if round(0,1)<epsilon:
            acao = env.action_space.sample()
        else:
            acao = np.argmax(q_table[estado])
            
        proximo_estado, recompensa, done, info = env.step(acao)
        
        q_antigo = q_table[estado, acao]
        proximo_maximo = np.max(q_table[proximo_estado])
        
        q_novo = (1 - alpha) * q_antigo + alpha * (recompensa + gamma * proximo_maximo)
        q_table[estado, acao] = q_novo
        
        if recompensa == -10:
            penalidades += 1
            
        estado = proximo_estado
    if i % 100 == 0:
        clear_output(wait=True)
        print('Episódio: ', i)
print('Treinamento concluído')

Episódio:  400
Treinamento concluído
CPU times: user 45.7 s, sys: 1.05 s, total: 46.8 s
Wall time: 45.6 s


## Avaliação do treinamento

In [6]:
total_penalidades = 0
episodios = 50
frames = []
for _ in range(episodios):
    estado = env.reset()
    penalidades, recompensa = 0, 0
    done = False
    while not done:
        acao = np.argmax(q_table[estado])
        estado, recompensa, done, info = env.step(acao)
        if recompensa == -10:
            penalidades += 1
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': estado,
            'action': acao,
            'reward': recompensa
        })
    total_penalidades += penalidades

In [8]:
from time import sleep
for frame in frames:
    clear_output(wait=True)
    print(frame['frame'])
    print('Estado', frame['state'])
    print('Ação', frame['action'])
    print('Recompensa', frame['reward'])
    sleep(.5)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

Estado 86
Ação 1
Recompensa -1


KeyboardInterrupt: 